In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
from tqdm import tqdm_notebook as tqdm
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.interactive(False)
matplotlib.use('Agg')

# find all of the files in the directory
import os
import gc

basePath= "gdrive/My Drive/Indian_birds/Dataset/"
melsPath= "gdrive/My Drive/Indian_birds/mels_class/"
os.listdir("gdrive/My Drive/Indian_birds/Dataset/")

birds=[] # list of all birds
for root, dirs, files in os.walk(basePath):
    if root == basePath:
        birds=dirs
print(birds)
birds50=[]                
flist=[] # list of all files
blist=[] # list of files for one bird 
i50=0;
count = 0
for i, bird in enumerate(birds):
    count +=1
    print(count)
    for root, dirs, files in os.walk(basePath+bird):
        for file in files:
            if file.endswith(".mp3"):
                blist.append(os.path.join(root, file))
                # print(blist)
    if len(blist) > 1:
        i50 = i50+1;
        print(i50, ". Found ", len(blist), ' files for ', bird ,'(',i+1,')')
        birds50.append(bird)
        flist.append(blist)
    blist=[]
print(birds50) 
print(root)

['Asian koel', 'Common tailorbird', 'White-throated kingfisher', 'White-cheeked barbet', 'Red-vented bulbul', 'Common hawk-cuckoo']
1
1 . Found  124  files for  Asian koel ( 1 )
2
2 . Found  126  files for  Common tailorbird ( 2 )
3
3 . Found  95  files for  White-throated kingfisher ( 3 )
4
4 . Found  94  files for  White-cheeked barbet ( 4 )
5
5 . Found  85  files for  Red-vented bulbul ( 5 )
6
6 . Found  84  files for  Common hawk-cuckoo ( 6 )
['Asian koel', 'Common tailorbird', 'White-throated kingfisher', 'White-cheeked barbet', 'Red-vented bulbul', 'Common hawk-cuckoo']
gdrive/My Drive/Indian_birds/Dataset/Common hawk-cuckoo


In [ ]:
def saveMel(signal, directory):
    gc.enable()
    # MK_spectrogram modified
    N_FFT = 1024         # length of fft window
    HOP_SIZE = 1024      # num of samples between successive rates
    N_MELS = 128         # number of Mel bands to generate
    WIN_SIZE = 1024      # window size amount of time over which waveform is sampled
    WINDOW_TYPE = 'hann' # The Hann window is a taper formed by using a raised cosine or sine-squared with ends that touch zero.
    FEATURE = 'mel'      # Defines a Melspectrogram
    FMIN = 1400          # lowest frequency (in Hz)

    fig = plt.figure(1,frameon=False)
    fig.set_size_inches(6,6)

    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    
    S = librosa.feature.melspectrogram(y=signal, sr=sr, #sampling rate
                                        n_fft=N_FFT,
                                        hop_length=HOP_SIZE, 
                                        n_mels=N_MELS, 
                                        htk=True, 
                                        fmin=FMIN, #high-pass filter freq.
                                        fmax=sr/2) # AMPLITUDE
    librosa.display.specshow(librosa.power_to_db(S**2,ref=np.max), fmin=FMIN) #power = S**2

    fig.savefig(directory)
    plt.ioff()
    plt.show(block=False)
    fig.clf()
    ax.cla()
    plt.clf()
    plt.close('all')

In [ ]:


size = {'desired': 5, # [seconds]
        'minimum': 4, # [seconds]
        'stride' : 0, # [seconds]
        'name': 5 # [number of letters]
       } # stride should not be bigger than desired length

print('Number of directories to check and cut: ', len(flist))

#step = (size['desired']-size['stride'])*sr 
# length of step between two cuts in seconds
step=1
if step>0:
    for bird, birdList in enumerate(flist):
        print("Processing ",bird,'. ', birds50[bird], "...")
        for birdnr, path in tqdm(enumerate(birdList)):
            # load the mp3 file
            directory=melsPath+str(bird)+birds50[bird][:size['name']]+"/"

            if not os.path.exists(directory):
                os.makedirs(directory)
                
            if not os.path.exists(directory+path.rsplit('/',1)[1].replace(' ', '')[:-4]+"1_1.png"):
                  
                signal, sr = librosa.load(path) # sr = sampling rate
                step = (size['desired']-size['stride'])*sr # length of step between two cuts in seconds
                
                nr=0;
                for start, end in zip(range(0,len(signal),step),range(size['desired']*sr,len(signal),step)):
                    # cut file and save each piece
                    nr=nr+1
                    # save the file if its length is higher than minimum
                    if end-start > size['minimum']*sr:
                        melpath=path.rsplit('/',1)[1]
                        melpath=directory+melpath.replace(' ', '')[:-4]+str(nr)+"_"+str(nr)+".png"
                        saveMel(signal[start:end],melpath)
                    #print('New file...',start/sr,' - ',end/sr)
                    #print('Start: ',start,'end: ', end, 'length: ', end-start)
                
            pass
else:    
    print("Error: Stride should be lower than desired length.")
    
print('Number of files after cutting: ')


Number of directories to check and cut:  6
Processing  0 .  Asian koel ...


0it [00:00, ?it/s]

Processing  1 .  Common tailorbird ...


0it [00:00, ?it/s]

Processing  2 .  White-throated kingfisher ...


...

Processing  4 .  Red-vented bulbul ...


0it [00:00, ?it/s]

Processing  5 .  Common hawk-cuckoo ...


0it [00:00, ?it/s]

Number of files after cutting: 


In [ ]:
# Test
import matplotlib.image as mpimg
from PIL import Image
ilist=[]
print(os.walk(melsPath))
for root, dirs, files in os.walk(melsPath):
    print(dirs)
    for file in files:
        if file.endswith(".png"):
            ilist.append(os.path.join(root, file))
print(len(ilist))
img=mpimg.imread(ilist[0])
imgplot = plt.imshow(img)
plt.show()
img=mpimg.imread(ilist[100])
imgplot = plt.imshow(img)
plt.show()
img=mpimg.imread(ilist[30])
imgplot = plt.imshow(img)
plt.show()
img=mpimg.imread(ilist[110])
imgplot = plt.imshow(img)
plt.show()

print("Found ",len(ilist)," files")

<generator object walk at 0x7fd6d4c21850>
['0Asian', '1Commo', '2White', '3White', '4Red-v', '5Commo']
[]
[]
[]
[]
[]
[]
4029
Found  4029  files
